In [1]:
csv_to_import = 'Scrubbed Card Swipes.csv'
output_file_suffix = 'Scrubbed Swipes'

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
initial_dataframe = pd.read_csv(csv_to_import)

values_to_drop = ['Contractor', 'Temp', 'Custodial', 'Visitor', 'NaN', 'zz']

initial_dataframe['Date'] = initial_dataframe['Event date/time'].transform(lambda x: x.split('T')[0])
initial_dataframe['Time'] = initial_dataframe['Event date/time'].transform(lambda x: x.split('T')[1])

initial_dataframe['Event date/time'] = initial_dataframe['Event date/time'].transform(lambda x:x.replace('T',' '))
initial_dataframe['Event date/time'] = initial_dataframe['Event date/time'].transform(lambda x:datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
initial_dataframe['Day of Week'] = initial_dataframe['Event date/time'].apply(lambda x: x.weekday())
initial_dataframe = initial_dataframe[initial_dataframe['Day of Week'].isin([5,6]) == False]

initial_dataframe['User name'] = initial_dataframe['User name'].transform(lambda x: str(x).replace(' - Mobile','').replace('-mobile','').replace(' - Fob','').replace(' - mobile', '').replace('-Mobile',''))


first_scan_frame = initial_dataframe.drop_duplicates(subset = ['User name', 'Date'], keep = 'first')
last_scan_frame = initial_dataframe.drop_duplicates(subset = ['User name', 'Date'], keep = 'last')

initial_dataframe.drop_duplicates(subset = ['User name', 'Date'], keep = 'last', inplace = True)

first_names_to_drop = []
for index, row in first_scan_frame.iterrows():
    for value in values_to_drop:
        if value in str(row['User name']):
            first_names_to_drop.append(row['User name'])
first_scan_frame = first_scan_frame[first_scan_frame['User name'].isin(first_names_to_drop) == False]

last_names_to_drop = []
for index, row in first_scan_frame.iterrows():
    for value in values_to_drop:
        if value in str(row['User name']):
            last_names_to_drop.append(row['User name'])
last_scan_frame = last_scan_frame[last_scan_frame['User name'].isin(last_names_to_drop) == False]

usernames_to_drop = []
for index, row in initial_dataframe.iterrows():
    for value in values_to_drop:
        if value in str(row['User name']):
            usernames_to_drop.append(row['User name'])

initial_dataframe = initial_dataframe[initial_dataframe['User name'].isin(usernames_to_drop) == False]

initial_dataframe.reset_index()

pivot_count_table = pd.pivot_table(data = initial_dataframe, values = 'User name', index = 'Date', aggfunc = 'count')

pivot_count_table.rename(columns = {'User name':'Employee Count'}, inplace=True)
pivot_count_table.to_csv('Employee Count by Day ' + output_file_suffix + '.csv')

In [4]:
initial_dataframe['is_here'] = True
pivot_is_here_table = pd.pivot_table(data = initial_dataframe, values = 'is_here', index = 'User name', columns = 'Date')
pivot_is_here_table.fillna('', inplace = True)
pivot_is_here_table.replace(to_replace = {True:'X'}, inplace = True)
pivot_is_here_table.to_csv('Employee By Date ' + output_file_suffix + '.csv')

In [5]:
first_scan_pivot = first_scan_frame.pivot(index = 'User name', columns = 'Date', values = 'Time')
last_scan_pivot = last_scan_frame.pivot(index = 'User name', columns = 'Date', values = 'Time')
merged_frame = first_scan_pivot.merge(last_scan_pivot, on = 'User name', suffixes = (' first',' last'), sort = True)
sorted_columns = merged_frame.columns.to_list()
sorted_columns.sort()
merged_frame = merged_frame[sorted_columns]

for index, i in enumerate(range(2, len(merged_frame.columns), 2)):
    merged_frame.insert(i+index, i,'---')
merged_frame.fillna('', inplace = True)

merged_frame.to_csv('First and Last Swipe '+output_file_suffix+'.csv')


c:\Users\Owen\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3364: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if (await self.run_code(code, result,  async_=asy)):
